# Exercise 3: Strings, Functions, If Else, For Loops, Git

In [1]:
import altair as alt
import numpy as np
import pandas as pd
from calitp_data_analysis import calitp_color_palette

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

* Using a `f-string`, load in your merged dataframe from Exercise 3.

In [ ]:
#GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/starter_kit/"

In [3]:
FOLDER = "../starter_kit/"
FILE_NAME = "merged.parquet"
m = pd.read_parquet(f"{FOLDER}{FILE_NAME}")
#m.head(2)

In [6]:
keep_col = ['ct_district', 'project_name', 'scope_of_work', 
                   'project_cost', 'lead_agency', 'overall_score']
sub_m = m[keep_col]

## Categorizing
* There are 40+ projects. They all vary in themes, some contain transit elements while others contain Active Transportation (ATP) components. Some contain both! 
* Categorizing data is an important part of data cleaning and analyzing so we can present the data on a more succinct level. 
* Let's organize projects into three categories.
    * ATP
    * Transit
    * General Lanes

### Task 1: Strings
* Below are some of the common keywords that fall into the categories detailed above. They are held in a `list`.
* Add other terms you think are relevant. 
* We are going to search the `Scope of Work` column for these keywords. 

In [7]:
#transit = ["transit", "passenger rail", "bus", "ferry"]
#atp = ["bike", "pedestrian", "bicycle", "sidewalk", "path"]
#general_lanes = ["general", "auxiliary", "highway"]

#### Step 1: Cleaning
* Remember in Exercise 2 some of the project names didn't merge between the two dataframes?
* In the real world, you won't have the bandwidth and time to replace each individual string value with a dictionary.
* An easy way to clean most of the values up is by lowercasing, stripping the white spaces, and replacing characters.
* We can search through a string column  easier when we simplify up the  values.

In [9]:
sub_m.scope_of_work = (
    sub_m.scope_of_work.str.lower() # Lowers the strings
    .str.strip() # Strips trailing white spaces
    .str.replace("-", " ") # Replaces hyphens with a space
    .str.replace("+", " ")
    .str.replace("_", " ")
)

/tmp/ipykernel_399/3540593799.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sub_m.scope_of_work.str.lower() # Lowers the strings
/tmp/ipykernel_399/3540593799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_m.scope_of_work = (


* `str.contains()` allows you to search through the column. 
* Let's search for projects that have "transit" in their descriptions. 
* There are many modifications you can make to `str.contains()`. Try them out and see what happens.
    * `df.loc[df.scope_of_work.str.contains("transit", case=False)]` 
        * Will search through your column without matching the case. It'll return rows with both "Transit" and "transit".
    * `df.scope_of_work.str.contains("transit", case=False, regex=False) `
        * Will return any matches that include `transit` rather than an exact match. It'll return rows with values like "transit" and "Transitory".

In [12]:
transit_only_projects = sub_m.loc[sub_m.scope_of_work.str.contains("transit")]
transit_only_projects.shape

(7, 6)

* Let's see how many transit projects are in this dataset.
* <b>Tip</b>
    * The data we typically work with tends to be wide (read about wide vs. long data [here](https://www.statology.org/long-vs-wide-data/)). Scrolling horizontally gets tiresome.
    * Placing all the columns you want to temporarily work within a `list` like `preview_subset` below is a good idea to temporarily narrow down your dataframe while working. 

In [13]:
preview_subset = ["project_name", "scope_of_work"]

In [14]:
transit_only_projects[preview_subset]

,project_name,scope_of_work
11,Greenway Gables Managed Lanes,"managed lanes prioritizing carpools, clean vehicles, and public transit, featuring real time traffic updates and incentives for sustainable transportation choices."
16,Sparkle City Smart Streets Initiative,"an intelligent transportation system integrating traffic management, real time transit information, and smart parking solutions to enhance mobility and reduce congestion."
19,Rolling Renaissance Rabbit Express,"new, eco friendly rolling stock for public transit, incorporating advanced propulsion systems, comfortable seating, and onboard amenities."
20,Transit Treasure Transit Oasis,"transit supportive features, including shelters, wi fi, and real time information displays, prioritizing passenger convenience and accessibility."
25,Trail of Treats and Transit Hub,"a multi use path connecting to public transit, featuring public art installations, wayfinding signage, and amenities like bike storage and repair stations."
27,Park and Ride Petal Paradise,"an attractive park and ride facility with amenities like ev charging, wi fi, and convenient access to nearby transit options."
43,Brookside Bus Blossom Lane,"prioritize public transportation and enhance air quality by dedicating lanes to buses and hovs on brookside boulevard, integrating smart traffic signals and real time transit information inspired by the ancient elves."


#### Step 2: Filtering
* We've found all the projects that says "transit" somewhere in its description. 
* Now there are just many more transit related elements to go. We forgot about bikes, bus, rail, so on and so forth.
* The method above leaves us with multiple dataframes. We actually just want our one original dataframe tagged with categories. 
* A faster way: join all the keywords you want into one large string.
    * | designates "or".
    * You can read `transit_keywords` as "I want projects that contain the word transit or passenger rail or bus or ferry"

In [27]:
transit = ["transit", "passenger rail", "bus", "ferry"]
atp = ["bike", "pedestrian", "bicycle", "sidewalk", "path"]
general_lanes = ["general", "auxiliary", "highway"]

In [16]:
transit_keywords = f"({'|'.join(transit)})"

In [17]:
# Print it out
transit_keywords

'(transit|passenger rail|bus|ferry)'

* Filter again - notice the .loc after df and how there are brackets around `df`?


In [18]:
transit = sub_m.loc[sub_m.scope_of_work.str.contains(transit_keywords)][preview_subset]
transit.shape

/tmp/ipykernel_399/1926078654.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  transit = sub_m.loc[sub_m.scope_of_work.str.contains(transit_keywords)][preview_subset]


(9, 2)

In [19]:
transit_only_projects[preview_subset].shape

(7, 2)

* Count how many more projects appear when we filter for 3 additional transit related keywords, compared to only transit below.

 2 more projects appeared


* Let's put this all together. 
* I want any project that contains a transit component to be tagged as "Y" in a column called  "Transit". 
* If a project doesn't have a transit component, it gets tagged as a "N".

In [20]:
sub_m["transit"] = np.where(
    (sub_m.scope_of_work.str.contains(transit_keywords)),
    "Y",
    "N",
)

/tmp/ipykernel_399/3163154953.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (sub_m.scope_of_work.str.contains(transit_keywords)),
/tmp/ipykernel_399/3163154953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_m["transit"] = np.where(


* Using `value_counts()` we can see the total of transit related vs non-transit related projects.

In [21]:
sub_m.transit.value_counts()

N    35
Y     9
Name: transit, dtype: int64

### Task 2: Functions 
* It looks like there are only  9 transit projects.
* We are missing the 2 other categories: ATP and General Lane related projects.
* We could repeat the steps above or we can use a **function.**
    * You can think of a function as a piece of code you write only once but reuse more than once.
    * In the long run, functions save you work and look neater when you present your work.
* You may not have realized this but you've been using functions this whole time.
    * When you are taking the `len()` you are using a built-in function to find the number of rows in a dataframe.

In [22]:
len(sub_m)

44

* `type` too is a built-in function that tells you what type of variable you are looking at. 

In [23]:
type(sub_m)

pandas.core.frame.DataFrame

In [26]:
#type(GCS_FILE_PATH)

In [25]:
type(transit)

pandas.core.frame.DataFrame

### Practice with outside resources
* Functions are incredibly important as such, **please spend more time than usual on this section and practice the tutorials linked.**
* [Tutorial #1 Practical Python for Data Science.](https://www.practicalpythonfordatascience.com/00_python_crash_course_functions)
* [DDS Functions.](https://docs.calitp.org/data-infra/analytics_new_analysts/01-data-analysis-intro.html#functions)

In [ ]:
# Practice here

####  Let's build a function together.
* This will be repetitive after the tutorials, but you will use functions all the time at DDS.
##### Step 1
* Start your function with `def` and the name you'd like. I'm calling it `categorize():`

In [ ]:
# def categorize():

##### Step 2 
* Now let's think of what are the two elements that we will repeat.
* We merely want to substitute `transit_keywords` with ATP or General Lane related keywords.
* Instead of the `df["Transit]"`, we want to create two new columns called something like `df["ATP]"` and `df["General_Lanes]"` to hold our yes/no results.
* Add the two elements that need to be substituted into the argument of your function.
    * It's good practice to specify what exactly the parameter should be: a string/list/dataframe/etc. 
    * Including this detail make it easier for your coworkers to read and use your code.

In [ ]:
# def categorize(df:pd.DataFrame, keywords:list, new_column:str):

##### Step 3
* It's also good to document what your function will return.
* In our case, it's a Pandas dataframe. 

In [ ]:
# def categorize(df:pd.DataFrame, keywords:list, new_column:str)->pd.DataFrame:

##### Step 4
* Think about the steps we took to categorize transit only.
* Add the sections of the code we will be reusing and sub in the original variables for the arguments.
    *  First, we joined the keywords from a list into a big string.
    *  Second, we searched through the Scope of Work column for the keywords.
    *  Third, if we find the keyword, we will tag the project as "Y" in the column "new_column". If the keyword isn't found, the project is tagged as "N".


In [28]:
def categorize(df: pd.DataFrame, keywords: list, new_column: str) -> pd.DataFrame:
    
    # Remember this used to be the list called transit_keywords, but it must be changed into a long string
    joined_keywords = f"({'|'.join(keywords)})" 

    # We are now creating a new column: notice how parameters has no quotation marks.
    df[new_column] = np.where((df.scope_of_work.str.contains(joined_keywords)), 
        "Y",
        "N",
    )

    # We are returning the updated dataframe from this function
    return df

#### Step 5 
* Now let's use your function: input the arguments in for each of the lists that hold the categorical keywords.

In [29]:
sub_m = categorize(df = sub_m, 
                keywords = atp, 
                new_column = "atp")
#sub_m.head(2)

/tmp/ipykernel_399/2245515441.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[new_column] = np.where((df.scope_of_work.str.contains(joined_keywords)),
/tmp/ipykernel_399/2245515441.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column] = np.where((df.scope_of_work.str.contains(joined_keywords)),


In [32]:
sub_m = categorize(df = sub_m, 
                keywords = general_lanes, 
                new_column = "general_lanes")
#sub_m.head(2)

/tmp/ipykernel_399/2245515441.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[new_column] = np.where((df.scope_of_work.str.contains(joined_keywords)),
/tmp/ipykernel_399/2245515441.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column] = np.where((df.scope_of_work.str.contains(joined_keywords)),


In [33]:
sub_m.shape

(44, 9)

In [34]:
sub_m.head(2)

,ct_district,project_name,scope_of_work,project_cost,lead_agency,overall_score,transit,atp,general_lanes
0,8,Meadow Magic Multi-Use Path,"a 2 mile class i bike lane and multi use path through a scenic meadow, featuring wildflower plantings, public art installations, and educational signage highlighting local wildlife.",4189348,Meadow Bunny Public Transportation (MBPT),82,N,Y,N
1,2,Bunny Hop Bike Boulevard,"a class ii bike lane with charming streetlights, benches, and bike racks designed to resemble carrot sticks, connecting residential neighborhoods to local schools and parks.",8647685,Unicorn Fairy Express Bus (UFX),65,N,Y,N


#### Check out your results
* Use the `groupby` technique from Exercise 2 to get some descriptive statistics for these 3 new columns
* Use `.reset_index()` after `aggregate()` to see what happens.
* Try `.reset_index(drop = True)` as well. 

In [36]:
pivot = (sub_m.groupby(['ct_district'])
        .agg({'transit': 'count', 
              'atp': 'count',
              'general_lanes' : 'count'}
             ).reset_index()
        )
print(pivot)

    ct_district  transit  atp  general_lanes
0             2        6    6              6
1             3        2    2              2
2             4        5    5              5
3             5        6    6              6
4             6        1    1              1
5             7        5    5              5
6             8        4    4              4
7             9        2    2              2
8            10        3    3              3
9            11        8    8              8
10           12        2    2              2


## Function + If-Else
* There are many cases in which we want to categorize our columns to create broader groups for summarizing and aggregating.
* Using a function with an If-Else clause will help us accomplish this goal.
* **<b>Resources</b>:**
    * [DDS Apply Docs](https://docs.calitp.org/data-infra/analytics_new_analysts/01-data-analysis-intro.html#functions)
    * [DDS If-Else Tutorial](https://docs.calitp.org/data-infra/analytics_new_analysts/01-data-analysis-intro.html#if-else-statements)
    
    

In [ ]:
# Practice here.

### Practice #1: 
* We are going to write an If-Else function that categorizes projects by whether it scored low, medium, or high based on its `overall_score` and percentiles.
    * If a project scores below the 25% percentile, it is a "low scoring project". If a project scores above the 25% percentile but below the 75% percentile, it is a "medium scoring project". Anything above the 75% percentile is "high scoring".
* In Data Science, we like to save our work into variables.
    * If new projects are added, then different percentiles will likely switch.
    * As such, you can save whatever percentile you like using `p75 = df.overall_score.quantile(0.75).astype(float)` which will change automatically when you load in the new data.
* Write an if-else and set the various percentiles using variables. 

In [39]:
p75 = sub_m.overall_score.quantile(0.75).astype(float)
p25 = sub_m.overall_score.quantile(0.25).astype(float)

In [41]:
def percentile(row):
    if row.overall_score <= p25:
        return 'low scoring project'
    elif row.overall_score >= p75:
        return 'high scoring project'
    else:
        return 'medium scoring project'

sub_m['score_percentile'] = sub_m.apply(percentile, axis = 1)
sub_m.head(2)

/tmp/ipykernel_399/3939735700.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_m['score_percentile'] = sub_m.apply(percentile, axis = 1)


,ct_district,project_name,scope_of_work,project_cost,lead_agency,overall_score,transit,atp,general_lanes,score_percentile
0,8,Meadow Magic Multi-Use Path,"a 2 mile class i bike lane and multi use path through a scenic meadow, featuring wildflower plantings, public art installations, and educational signage highlighting local wildlife.",4189348,Meadow Bunny Public Transportation (MBPT),82,N,Y,N,high scoring project
1,2,Bunny Hop Bike Boulevard,"a class ii bike lane with charming streetlights, benches, and bike racks designed to resemble carrot sticks, connecting residential neighborhoods to local schools and parks.",8647685,Unicorn Fairy Express Bus (UFX),65,N,Y,N,medium scoring project


In [42]:
sub_m.score_percentile.value_counts()

medium scoring project    21
low scoring project       12
high scoring project      11
Name: score_percentile, dtype: int64

### Practice #2
* Goal:
    * Above, we can see all types of combinations of categories a project can fall into. 
    * Let's do away with these "Y" and "N" columns and create actual categories in an actual column called `categories`.
    * If a project has "N" for all 3 of the General Lane, Transit, and ATP columns, it should be `Other`. 
    * If a project has "Y" for all 3, it should be categorized as "General Lane, Transit, and ATP".
    * If a project has "Y" for only ATP and Transit, it should be categorized as "Transit and ATP".
    * Yes this will be very tedious given all the combinations!
* Resource:
    * [Geeks for Geeks: if-else with multiple conditions](https://www.geeksforgeeks.org/check-multiple-conditions-in-if-statement-python/)

In [44]:
def project_type(row):
    if row.transit == 'N' and row.atp == 'N' and row.general_lanes == 'N':
        return 'Other'
    elif row.transit == 'Y' and row.atp == 'Y' and row.general_lanes == 'Y':
        return 'General Lane, Transit, and ATP'
    elif row.transit == 'Y' and row.atp == 'Y' and row.general_lanes == 'N':
        return 'Transit and ATP'
    elif row.transit == 'Y' and row.atp == 'N' and row.general_lanes == 'Y':
        return 'General Lane and Transit'
    elif row.transit == 'N' and row.atp == 'Y' and row.general_lanes == 'Y':
        return 'General Lane and ATP'
    elif row.transit == 'Y' and row.atp == 'N' and row.general_lanes == 'N':
        return 'Transit'
    elif row.transit == 'N' and row.atp == 'Y' and row.general_lanes == 'N':
        return 'ATP'
    elif row.transit == 'N' and row.atp == 'N' and row.general_lanes == 'Y':
        return 'General Lane'

sub_m['categories'] = sub_m.apply(project_type, axis = 1)
sub_m.head(2)

/tmp/ipykernel_399/269874075.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_m['categories'] = sub_m.apply(project_type, axis = 1)


,ct_district,project_name,scope_of_work,project_cost,lead_agency,overall_score,transit,atp,general_lanes,score_percentile,categories
0,8,Meadow Magic Multi-Use Path,"a 2 mile class i bike lane and multi use path through a scenic meadow, featuring wildflower plantings, public art installations, and educational signage highlighting local wildlife.",4189348,Meadow Bunny Public Transportation (MBPT),82,N,Y,N,high scoring project,ATP
1,2,Bunny Hop Bike Boulevard,"a class ii bike lane with charming streetlights, benches, and bike racks designed to resemble carrot sticks, connecting residential neighborhoods to local schools and parks.",8647685,Unicorn Fairy Express Bus (UFX),65,N,Y,N,medium scoring project,ATP


In [45]:
sub_m.categories.value_counts()

Other                   15
ATP                     11
Transit                  8
General Lane             7
General Lane and ATP     2
Transit and ATP          1
Name: categories, dtype: int64

### Please export your output as a `.parquet` to GCS before moving onto the next step

In [46]:
FOLDER = "../starter_kit/"
FILE_NAME = "project_category.parquet"

sub_m.to_parquet(f"{FOLDER}{FILE_NAME}")

## For Loops 
* For Loops are one of the greatest gifts of Python. 
* It runs code from the beginning to the end of a list. 
* Below is a simple for loop that prints out all the numbers in range of 10.


In [ ]:
for i in range(10):
    print(i)

* Here, I'm looping over a couple of columns in my dataframe and printing some descriptive statistics about it.
* Notice how I have to use `print` and `display` to show the results.
    * Try this same block of code without `print` and `display` to see the difference.

In [ ]:
for column in ["zev_score", "vmt_score", "accessibility_score"]:
    print(f"Statistics for {column}")
    display(df[column].describe())

### Practice using a for loop
* I have aggregated the dataframe for you.

In [47]:
agg1 = (
    sub_m.groupby(["categories"])
    .aggregate(
        {"overall_score": "median", "project_cost": "median", "project_name": "nunique"}
    )
    .reset_index()
    .rename(
        columns={
            "overall_score": "median_score",
            "project_cost": "median_project_cost",
            "project_name": "total_projects",
        }
    )
)

In [48]:
agg1

,categories,median_score,median_project_cost,total_projects
0,ATP,68.00,6566728.00,11
1,General Lane,78.00,5133341.00,7
2,General Lane and ATP,71.00,5575973.00,2
3,Other,66.00,3321300.00,15
4,Transit,75.00,3658785.00,8
5,Transit and ATP,63.00,5670780.00,1


* I have also prepared an Altair chart function. 

In [49]:
def create_chart(df: pd.DataFrame, column: str) -> alt.Chart:
    title = column.replace("_", " ").title()
    chart = (
        alt.Chart(df, title=f"{title} by Categories")
        .mark_bar(size=20)
        .encode(
            x=alt.X(column),
            y=alt.Y("categories"),
            color=alt.Color(
                "categories",
                scale=alt.Scale(
                    range=calitp_color_palette.CALITP_CATEGORY_BRIGHT_COLORS
                ),
            ),
            tooltip=list(df.columns),
        )
        .properties(width=400, height=250)
    )
    return chart

* Use the function above to create a chart out of the aggregated dataset.

In [53]:
create_chart(df = agg1,  
             column = "median_score")

alt.Chart(...)

In [54]:
create_chart(df = agg1,  
             column = "median_project_cost")

alt.Chart(...)

In [55]:
create_chart(df = agg1,  
             column = "total_projects")

alt.Chart(...)


* We have a couple of other columns left that still need to be visualized. 
* This is the perfect case for using a for loop, since all we want to do is replace the column above with the two remaining columns. 
* Try this below! 
    * You'll have to create a `list` that contains the rest of the columns.
    * You'll have to wrap the function with `display()` to get your results.

In [57]:
agg_list = ['median_score', 'median_project_cost', 'total_projects']
for col in agg_list:
    visual = create_chart(df = agg1,  
             column = col)
    display(visual)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## GitHub - Pull Requests
* In Exercise 1, you created a new branch that you are working on now.
* Now that you are done with Exercise 3, you are at a nice stopping point to commit your work to our `main` branch.

**Steps**
1. Do the normal workflow of `committing` your work. 
2. Navigate to the our `data-analyses` [repo over here](https://github.com/cal-itp/data-analyses).
3. Follow the steps detailed in [this video](https://youtu.be/nCKdihvneS0?si=nPlBOAMcgO1nv3v1&t=95). 
4. Once you're done writing, scroll down the bottom and click `merge pull request` 
<img src= "./starter_kit_img.png">
5. Your work is now merged into the `main` branch of our `data-analyses` repo. 
6. To check, navigate to the our [repo](https://github.com/cal-itp/data-analyses) and to this `starter_kit` folder to make sure your notebooks are on the `main` branch.
7. Delete the branch `your_branch`. 
    * It's considered outdated now because your changes are on the `main branch`. In the terminal, paste `git branch -d your_branch`. 
    * If that doesn't work, paste `git branch -D your_branch`.
8. Continuing in the terminal, paste `git switch main`. 
9. Paste `git pull origin main`. 
    * This pulls down the work you just uploaded, along with the other work your coworkers have committed onto the main branch. 
9. Create a new branch `git switch -c your_branch` to continue working on exercises 4 and 5.
    * Your new branch can have the same name as the branch you just merged in.